# Map, Filter and Fold (Reduce) Operations

In many languages, the use of for-loops/while loops to iterate is replaced by operations on data structures such as `map`, `filter` and `fold`. In this lecture, we provide a brief overview with some examples. We show how many varieties of loops or equivalently recursion, can be systematically replaced by these operations.

## Functions as First Class Objects in Scala

The main idea behind functional programming is that `functions are first class objects`. What this means is that just like integers, strings, lists and other data, functions can be passed around. 
1. They can be used as arguments to other functions.
2. Functions can be created from inside a function and returned as a value.
3. You can have variables (vars) that can be assigned to functions.
4. and many more ideas that we will revisit later...


### Idea 1: You can pass functions as arguments to other functions

In [1]:
def callTwice(f: Int => Int, arg: Int) = f(f(arg))

defined function callTwice

The function `callTwice` takes in two arguments, `f` a function of type `Int => Int` and `arg` an argument of type `Int` 

- What does `Int => Int` mean? It annotates a function type in scala. I.e, a function that takes in an Int argument and returns an Int argument.

In [3]:
def addOne(x: Int) = x + 1
val y = callTwice(addOne, 25)
println(y)

27


defined function addOne
y: Int = 27

In [9]:
def factorialHelper(x: Int, acc:Int= 1): Int = 
    if (x <= 0) acc
    else factorialHelper(x-1, acc*x)

def factorial(x: Int): Int = factorialHelper(x, 1)

val y = callTwice(factorial, 3)
val z = callTwice(factorial, 4)

defined function factorialHelper
defined function factorial
y: Int = 720
z: Int = -775946240

In [13]:
def filterList(l: List[Int], f: Int => Boolean): List[Int] = {
    // f is a function from Int to Boolean
    // l is a list
    // make a new list of only those elements for which f returns true
    var newList: List[Int] = List()
    for (x <- l){
        if (f(x)) 
            newList = newList ++ List(x) // Append x to the end
    }
    return newList
}

def isOdd(x: Int): Boolean = x %2 == 1
filterList(List(1, 3, 12, 12, 15, 18, 19, 13, 20, 22, 19, 14), isOdd)

defined function filterList
defined function isOdd
res12_2: List[Int] = List(1, 3, 15, 19, 13, 19)

### Idea 2: You can create and return a function from inside another function.


In [14]:
def bindSecondArgument(f: (Int, Int) => Int, arg2: Int ): (Int=> Int) = {
    def g(x: Int): Int = f(x, arg2)
    return g
}

defined function bindSecondArgument

Function `bindSecondArgument` is quite interesting. It inputs two arguments: 
- f is a function from two integers to an integer and 
- arg2 is an integer
We define a function `g` with argument `x` that calls `f(x, arg2)`
The function `g` is returned by `bindSecondArgument`

In [19]:
def foo(x: Int, y: Int) = x * y

val timesThree = bindSecondArgument(foo, 3) // You can use def here but val is also fine

timesThree(4)

defined function foo
timesThree: Int => Int = ammonite.$sess.cmd13$Helper$$Lambda$2615/1454492182@388feb9a
res18_2: Int = 12

In [21]:
def consecuteArguments(f: (Int, Int, Int) => Int) : Int => Int = {
    def g(x: Int) = f(x, x+1, x+2)
    return g
}

defined function consecuteArguments

In [26]:
def times3(x:Int, y: Int, z: Int) = x*y*z
val g = consecuteArguments(times3)
println(g(5))
println(g(20))

210
9240


defined function times3
g: Int => Int = ammonite.$sess.cmd20$Helper$$Lambda$2662/1302964374@6e8b9af8

### Anonymous functions

In [27]:
def foo(x: Int) = x + 10

defined function foo

In [29]:
val foo_anon: Int => Int = ( x => x + 10 ) 

foo_anon: Int => Int = ammonite.$sess.cmd28$Helper$$Lambda$2991/664546194@7729f897

In [30]:
def bar(x: Int, y: String) = x match {
    case 0 => "zero"
    case 1 => "one"
    case _ => y + "~~> " + x
}

defined function bar

In [31]:
val bar_anon: (Int, String) => String = 
(x, y) => {
    x match {
        case 0 => "zero"
        case 1 => "one"
        case _ => y + "~~> " + x
    }
}

bar_anon: (Int, String) => String = ammonite.$sess.cmd30$Helper$$Lambda$3028/1964167843@5b4156d8

In [32]:
bar(10, "hello")

res31: String = "hello~~> 10"

In [33]:
bar_anon(10, "hello")

res32: String = "hello~~> 10"

In [34]:
bar(1, "hi")
bar_anon(1, "hi")

res33_0: String = "one"
res33_1: String = "one"

Anonymous functions are useful inside code where they allow us to succinctly create new functions and pass them.
Do not worry if the strange factorial function below seems somewhat less intuitive. We will learn all about this within a couple of weeks. It is written in a style called the "Continuation Passing Style".


In [39]:
def strange_factorial_fun(x: Int, k: Int => Int): Int = {
    if (x <= 0) k(1)
    else
        factorial_fun(x-1, v => { k(x*v) })
}

defined function strange_factorial_fun

In [38]:
println(factorial_fun(5, x => x))
println(factorial_fun(6, x => x))

120

res37_1: Int = 720

One useful feature of an anonymous function is you can directly pattern match on the input argument.

In [40]:
val f: Int => String = {
    case 0 => "zero"
    case 1 => "one"
    case x if x >= 2 => x.toString
    case x if x < 0 => "minus: " + x.toString
}

f: Int => String = ammonite.$sess.cmd39$Helper$$Lambda$3078/161432483@47a7b026

In [42]:
f(10)
f(1)
f(-1)

res41_0: String = "10"
res41_1: String = "one"
res41_2: String = "minus: -1"

In [44]:
sealed trait List
case object Nil extends List
case class ConsNum(j: Int, l: List) extends List
case class ConsList(l1: List, l2: List) extends List

defined trait List
defined object Nil
defined class ConsNum
defined class ConsList

In [46]:
val nestingDepthAnon: List => Int = {
    case Nil => 0
    case ConsNum(_, l) => nestingDepth(l)
    case ConsList(l1, l2) => math.max(1+nestingDepth(l1), nestingDepth(l2))
}

nestingDepthAnon: List => Int = ammonite.$sess.cmd45$Helper$$Lambda$3342/746381704@65930c05

In [48]:
def nestingDepth(lst: List): Int= lst match {
    case Nil => 0
    case ConsNum(_, l) => nestingDepth(l)
    case ConsList(l1, l2) => math.max(1+nestingDepth(l1), nestingDepth(l2))
}

defined function nestingDepth

In [49]:
val nestingDepthAnonAlternative: List => Int = lst => {
    lst match {
    case Nil => 0
    case ConsNum(_, l) => nestingDepth(l)
    case ConsList(l1, l2) => math.max(1+nestingDepth(l1), nestingDepth(l2))
}
}

nestingDepthAnonAlternative: List => Int = ammonite.$sess.cmd48$Helper$$Lambda$3346/722366535@50b94457

## Map operation

The idea of a map operation is to apply a function $f$ to every member of a container (eg., list, array, map, etc.) and return a new container.

### Example 1

We have a list `List(1, 3, 4, 5, 6, 110, 12, 2)`. We wish to compute the square of each element in the list and make a new list with the result.

In [5]:
def recursivelySquareEachElt(l: List[Int], acc: List[Int] ): List[Int] = {
    if (l.length == 0)
        acc.reverse
    else
        recursivelySquareEachElt(l.tail, (l.head*l.head)::acc)
}

defined function recursivelySquareEachElt

In [6]:
recursivelySquareEachElt(List(10), List())

res5: List[Int] = List(100)

In [7]:
recursivelySquareEachElt(List(1, 3, 4, 5, 6, 110, 12, 2), List())

res6: List[Int] = List(1, 9, 16, 25, 36, 12100, 144, 4)

Using the map operator over lists.

In [10]:
def squareEachElt(l: List[Int]): List[Int] =  l.map( x => x*x ) 
// x => x * x is an anonymous function that squares its arguments.

defined function squareEachElt

In [9]:
squareEachElt(List(1, 3, 4, 5, 6, 110, 12, 2))

res8: List[Int] = List(1, 9, 16, 25, 36, 12100, 144, 4)

`l.map(f)` says that apply the function `f` on each element of the list `f`.

First of all, the elements of the lists must be some type `A`, let's say. 
Next, the function `f` must be of type `A => B`.

Last but not least, `l.map(f)` applies `f` to every element in the list and returns a new list
of type `B`.

In [11]:
def sayHelloTo(l: List[String]): List[String] = l.map( x => ("Hello "+ x))

defined function sayHelloTo

In [12]:
sayHelloTo(List("Cat", "Dog", "World"))

res11: List[String] = List("Hello Cat", "Hello Dog", "Hello World")

## Sum up all squares of numbers from 1 to n 


In [17]:
def sumUpto(n: Int)= ((1 to n).map(x => x * x)).sum
// (1 to n) creates a range or squence (not a list) from 1 to n.
// I can apply map on this to square each element.
// Calling sum on the result sums it up.
// Anonymous function x=> .x * x does the squaring.

defined function sumUpto

In [15]:
sumUpto(3)
sumUpto(10)
sumUpto(25)

res14_0: Int = 14
res14_1: Int = 385
res14_2: Int = 5525

## Filter Operation.

Just like we have used map to apply a function to each element and make a new container, we use `filter` to remove all elements that do not satisfy a predicate.

__Predicate__ A preducate is a funciton that takes in a value and returns true/false.

`l.filter(c)` filters all those elements that do not satisfy the condition `c` from the list `l`.

In [18]:
def retainAllMultiplesOfThree(l: List[Int]): List[Int] = {
    l.filter( x => x%3 == 0 )
}

defined function removeAllMultiplesOfThree

In [19]:
removeAllMultiplesOfThree(List(10, 15, 18, 12, 3, 1, 5, 7, 8, 14))

res18: List[Int] = List(15, 18, 12, 3)

In [23]:
// Sum up all odd squares from 1 to n
def sumOddSquares(n : Int): Int = {
    (1 to n).filter(x => x%2 == 1).map(x => x * x).sum
    // (1 to n) Range of all numbers frmo 1 to n, not a List
    // filter -- take away odd numbers. 
    // Map from x to x* x
    // sum up the answer
}

defined function sumOddSquares

In [22]:
sumOddSquares(10)

res21: Int = 165

## Fold Operations

Fold/reduce operations are useful to gather all data thus far during a computation. Take a list

$$[l_1, l_2, \ldots, l_n] $$.

We wish to sum up the numbers in the list.
This is achieved in a loop with accumulator.
~~~
acc = 0
for each item in List
   acc = acc + item
return acc
~~~

We can also do it with fold left operator.





In [25]:
def sumList(l: List[Int]): Int = l.foldLeft (0) ((acc, x) => acc + x )
// Fold left with initial value of accumulator = 0
// Every time we have a new list element x and accumulator value acc, update acc by acc + x

defined function sumList

In [26]:
sumList(List(1, 2, 3,4, 5, 6, 7, 8, 9, 10))

res25: Int = 55

The expression

~~~
l.foldLeft (initialValue) (function)
~~~

replaces the loop

~~~
var acc = initialValue // Start acc with initial value 
for elt in l 
    acc = function(acc, elt) // call function on acc as first arg and the list elt as second.
return acc
~~~